In [ ]:
# CNN : Convolutional Neural Network
# 2차원 합성곱
# 오른쪽으로 한칸 이동, 아래로 한칸이동
# 4*4, 3*3, 2*2

# 3,1,0,7           w1,w2,w3               3*w1 + 1*w2, 0*w3 + 6*w4 + 4*w5 + 8*w6 + 4*w7 + 5 *w8 + 1*w9 + b => 1개의 출력 
# 6,4,8,2           w4,w5,w6
# 4,5,1,1           w7,w8,w9
# 3,2,5,8


# 인공 신경망은 처음에 가중치 w1~w9와 절편 b를 랜덤하게 초기화한 다음 epochs를 반복하면서 경사 하강법 알고리즘을
# 사용하여 손실이 낮아지도록 최적의 가중치와 절편을 찾아갑니다. 이것이 모델 훈련입니다.

# CNN은 입력데이터 전체에 가중치를 적용하는 것이 아니라 일부에 가중치를 곱한다.

# CNN 계산을 통해 얻은 출력을 특별히 특성맵(feature map)이라고 한다.
# kernel_size는 하이퍼파라미터이다, 보통 (3,3) or (5,5)를 많이 사용함.

#from tensorflow import keras
#keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu')

# padding : filter의 슬라이딩의 숫자를 늘려주는 것, 입력 배열의 주위를 가상의 원소로 채우는 것
# 픽셀마다 기여하는 정보가 틀림. 주변 픽셀의 정보를 잘 감지하기 위해 추가함

#keras.layers.Conv2D(10, kernel_size(3,3), activation='relu', padding='same') # padding='valid' 

# 스트라이드 : 이동하는 크기를 스트라이드라고 함
#keras.layers.Conv2D(10, kernel_size(3,3), activation='relu', padding='same', strides=1) 

# pooling, 가중치는 없다, 겹치지 않는다
# 평균풀링
# 최대풀링

#keras.layers.MaxPooling2D(2)


# 6*6 => 3*3의 3개의 필터 => (4,4) 특성맵 3개 => (4,4,3) 특성맵 => 2*2 최대풀링 => (2,2,3) 특성맵 => 12, 크기입력

# 3차원(너비*높이*깊이(채널)) 합성곱
# 입력의 깊이와 커널의 깊이는 같다.

#(4,4,3)크기 입력과 (3,3,3)크기 커널의 합성곱

import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

(train_input, train_target), (test_input, test_target) =\
 keras.datasets.fashion_mnist.load_data()

print(train_input.shape, train_target.shape)


#keras model create
from sklearn.model_selection import train_test_split
(train_input, train_target), (test_input, test_target) =\
 keras.datasets.fashion_mnist.load_data()

#채널 차원을 늘림
train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0

#keras model create
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)


model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(28,28,1)))
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()


keras.utils.plot_model(model, show_shapes=True, to_file='cnn-architecture.png', dpi=300)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5')

early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_scaled, train_target, epochs=20,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

model.evaluate(val_scaled, val_target)

plt.imshow(val_scaled[0].reshape(28,28), cmap='gray_r')
plt.show()

preds = model.predict(val_scaled[0:1])
print(preds)

plt.bar(range(1,11), preds[0])
plt.xlabel('class')
plt.ylabel('prob.')
plt.show()

classes = ['티셔츠', '바지', '스웨터', '드레스', '코트', '샌달', '셔츠', '스니커즈', '가방', '앵클 부츠']
test_scaled = test_input.reshape(-1, 28, 28, 1) / 255.0

model.evaluate(test_scaled, test_target)


(60000, 28, 28) (60000,)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               313700    
_________________________________________________________________
dropout_6 (Dropout)          